### Прочитали fasta формат

In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("sequence.fasta", format="fasta")


In [37]:
record.seq[0:127]

Seq('MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYET...DIT', SingleLetterAlphabet())

### Рассмотрим результат BLAST поиска

In [173]:
result_handle = open('W8ZR3WEN015-Alignment.xml')

In [174]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

In [175]:
good_almns = {}
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            titles = alignment.title.split('>')
            for title in titles:
                t = title.split('|')
                ss =['517848989']
                for s in ss:
                    if t[1] == s:
                        print("****Alignment****")
                        print("sequence:", alignment.title)
               


Варианты, которые получлсь у меня и совпали со статьей 504822023, 505222758, 

In [1]:
good_almns

In [2]:
"""for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print(hsp.query_start, hsp.query_end, sep = ' ')
            print(hsp.query[0:75] + '...')"""

'for blast_record in blast_records:\n    for alignment in blast_record.alignments:\n        for hsp in alignment.hsps:\n            print("****Alignment****")\n            print("sequence:", alignment.title)\n            print(hsp.query_start, hsp.query_end, sep = \' \')\n            print(hsp.query[0:75] + \'...\')'

### Скачивание OneKP

In [4]:
from bs4 import BeautifulSoup   # Для обработки HTML
import requests # для получения кода страницы

Пропарсила страницу нашла нужные ссылки

In [5]:
data = requests.get('http://www.onekp.com/public_data.html')

In [9]:
soup = BeautifulSoup(data.text, 'html.parser')

In [1]:
print(soup.prettify())

Получим все ссылки

In [28]:
s= []
for string in soup.find_all('a'):
    s.append(string.get('href'))

Будем обращаться для скачивания по необходимым ссылкам

In [57]:
import urllib

In [55]:
res = [elem if elem.find("Trans-assembly")!=-1 else None for elem in s]

Попытка (удачная) скачать файл

In [59]:
url = 'http://206.12.96.204/onekp/URDJ-SOAPdenovo-Trans-assembly.fa.bz2'
urllib.request.urlretrieve(url, "URDJ.bz2")

('URDJ.bz2', <http.client.HTTPMessage at 0x7f476baf3e10>)

### Разделение файла с белками из Interpro

In [3]:
with open('proteins-matched_interpro.txt', 'r') as file:
    n = 0
    s_1, s_2, s_3, s_4 = [], [], [], []
    for line in file:
        if line.startswith('>'):
            n += 1
        if n <= 98328:
            s_1.append(line)
        elif 98328 < n <= 98328*2:
            s_2.append(line)
        elif 98328 * 2 < n <= 98328*3:
            s_3.append(line)
        else:
            s_4.append(line)
            
            

In [4]:
with open('protein_1.txt','w') as file:
    file.write(''.join(s_1))
with open('protein_2.txt','w') as file:
    file.write(''.join(s_2))
with open('protein_3.txt','w') as file:
    file.write(''.join(s_3))
with open('protein_4.txt','w') as file:
    file.write(''.join(s_4))

### Парсинг файлов mast и meme 

In [89]:
from bs4 import BeautifulSoup
with open("Without_Cys/mast_interpro_1_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')
#print(soup.prettify())

In [90]:
motifs = soup.motifs.findAll('motif')

In [91]:
length = [motif['length'] for motif in motifs]

In [92]:
length

['35', '31', '29']

На 40 месте  в мотиве стоит **С** на который и нужно проверить или для Cysless на 24 месте не должен стоять.

In [93]:
sequences = soup.sequences.findAll('sequence')

In [94]:
import re
reg = re.compile('[^a-zA-Z ]')

Нашли все имена таких сиквенсов и также запсали данные о них в формати xml

In [95]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [59]:
with open('Without_Cys/Cysless_sequences_4.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

Все отсеившееся по нашему выбору сиквенсы запишем в отдельный файл

In [124]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [125]:
new_file = []
for name in Cys_seq_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [126]:
with open('Cysless_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))

### Парсинг результатов pfam 

Сначала вручную удаляем информацию с *#* кроме последнего, с наименованием величин.  

In [47]:
import pandas as pd

In [115]:
columns = []
d = []
with open('Without_Cys/pfam_4_cl.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [116]:
data = pd.DataFrame(d, columns=columns)

In [117]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,M1FCN8,32,128,22,129,PF13426.7,PAS_9,Domain,11,103,104,76.6,1.5e-21,1,CL0183
1,M1FCN8,148,251,148,251,PF13426.7,PAS_9,Domain,1,104,104,64.4,9.1e-18,1,CL0183
2,M1FCN8,265,416,263,416,PF00990.21,GGDEF,Domain,3,161,161,125.5,1.5e-36,1,CL0276
3,V4Y023,135,236,134,236,PF13426.7,PAS_9,Domain,2,104,104,42.9,4.7e-11,1,CL0183
4,V4Y023,257,361,255,361,PF13426.7,PAS_9,Domain,3,104,104,76.0,2.3e-21,1,CL0183


### Идентификация LOV доменов

#### 1. Отбор по цистеину в первом мотиве

##### а) С цистеином

См. **Парсинг mast и meme**

##### б) Без цистеина

--//--

#### Отбор по P-value для каждого мотива и по нахождению в PAS fold

#### а) С Cys

In [175]:
from bs4 import BeautifulSoup
with open("With_Cys/Cys_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [176]:
sequences = soup.sequences.findAll('sequence')

Проверим, что хотя бы какой-то сегмент имеет мотивы с нужным нам P-value и находятся в PAS fold

In [177]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [82]:
seq_data = data.loc[data['seq id'] == 'A0A126WYA8']
df = seq_data.loc[seq_data['clan'] == 'CL0183']
df

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
17843,A0A126WYA8,138,233,130,234,PF13426.7,PAS_9,Domain,8,103,104,59.4,3.3e-16,1,CL0183
17844,A0A126WYA8,419,511,410,514,PF13426.7,PAS_9,Domain,9,101,104,71.4,6e-20,1,CL0183


In [178]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k]
    df = seq_data.loc[seq_data['clan'] == 'CL0183']
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <=5 and hit[1] + 43 - int(df.iloc[i, 4]) <= 1:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)    

In [181]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [182]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [183]:
with open('With_Cys/final_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))

#### б) без Cys

In [125]:
from bs4 import BeautifulSoup
with open("Without_Cys/Cysless_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [126]:
sequences = soup.sequences.find_all('sequence')

In [127]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if (bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True and bool(seg.find(idx = '2')) == True):
            hits = seg.find_all('hit')
            if (float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15 and float(hits[2]['pvalue']) <= 1e-15):
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos']), 
                                                         int(hits[2]['pos'])])


In [128]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k]
    df = seq_data.loc[seq_data['clan'] == 'CL0183']
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <= 2 and hit[2] + 28 - int(df.iloc[i, 4]) <= 10:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)    

In [129]:
len(Cys_seq_names)

839

In [130]:
len(seqs)

146

In [131]:
len(new_names)

146

In [132]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [133]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [134]:
with open('Without_Cys/final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

In [41]:
with open('With_Cys/final_proteins.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [42]:
with open('Without_Cys/final_proteins_cl.fasta', 'r') as f:
    fasta_cl = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta_cl.setdefault(name, []).append(line)

In [43]:
repeat = list(fasta.keys() & fasta_cl.keys())
len(repeat)

0

Нашлись пересечения по наименованиям в final_proteins и final_proteins_cl удалим дубликаты, и проверим к какому именно классу они относятся

In [35]:
for r in repeat:
    fasta_cl.pop(r)

In [39]:
new_file = []
for name in fasta_cl.keys():
    new_file.append(name + '\n')
    for s in fasta_cl.get(name):
        new_file.append(s + '\n')

In [40]:
with open('Without_Cys/final_proteins_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

 В итоге получили файл final без повторов, и уменьшили количество сиквенсов в final_cl.

Теперь рассмотрим эффекторы, используя данные pfam полученные ранее

### Эффекторы LOV

#### C Cys

In [49]:
columns = []
d = []
with open('With_Cys/pfam_1.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [50]:
data = pd.DataFrame(d, columns=columns)

In [51]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,A0A0J6VGA8,62,156,52,157,PF13426.7,PAS_9,Domain,9,103,104,69.7,2.1e-19,1,CL0183
1,A0A0J6VGA8,175,235,173,237,PF00512.25,HisKA,Domain,3,64,67,34.0,2.2e-08,1,CL0025
2,A0A0J6VGA8,280,402,279,405,PF02518.26,HATPase_c,Domain,2,109,112,49.2,6.5e-13,1,CL0025
3,A0A0J6VGA8,427,537,427,538,PF00072.24,Response_reg,Domain,1,111,112,67.4,1.1e-18,1,CL0304
4,A0A059UGE7,331,427,324,428,PF13426.7,PAS_9,Domain,7,103,104,69.9,1.9e-19,1,CL0183


In [52]:
from bs4 import BeautifulSoup
with open("With_Cys/Cys_sequences_1.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [53]:
sequences = soup.sequences.findAll('sequence')

Для каждого наименования в final-ах, будем рассматривать расположение LOV (по наличию двух мотивов внутри) и ближайшие к нему эффекторы